# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '4h'
higher_timeframe = '1d'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'vol_window': [24], # [6, 12, 24, 48, 72, 96],
                            'vol_q_high': [0.6], # np.arange(0.1, 1, 0.05), 
                            'vol_q_low': [0.1], # np.arange(0.1, 1, 0.05),
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.85], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.75], # np.arange(0.5, 1, 0.05)
                            },
                'Trend': {
                            'timeperiod': np.arange(2, 50, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1d, trade type is buy
Number of combinations is 24


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [02:50<00:00,  7.12s/it]


# Check local statistics

In [ ]:
# result
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

In [2]:
# vol_q
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
180,Pattern_Trend,48,0.60,0.10,1,5,0.85,0.75,16,0,52,1.773813,3.930417,40.238250,204.381667,1.3216,1.6495,1.7736,1.7501,1.7265,1.8634,1.9259,1.9853,2.0126,1.9579,2.0429,2.0072,2.0140,1.8585,1.7828,1.7903,1.7822,1.7074,1.6353,1.6206,1.5557,1.5610,1.6032,1.6440,0.38,0.79,1.16,1.52,1.87,2.27,2.77,3.25,3.66,4.02,4.39,4.69,4.88,5.03,5.18,5.28,5.31,5.59,5.57,5.58,5.58,5.42,5.19,4.95
181,Pattern_Trend,48,0.60,0.15,1,5,0.85,0.75,16,0,63,1.846817,3.733750,53.349450,235.226250,1.3343,1.7152,1.9864,1.9699,1.9413,2.0765,2.1357,2.0970,2.0502,1.9785,2.0369,1.9980,2.0092,1.9646,1.8964,1.8217,1.7856,1.7265,1.6685,1.6558,1.6003,1.5943,1.6268,1.6540,0.32,0.72,1.11,1.49,1.85,2.25,2.74,3.20,3.59,3.91,4.22,4.49,4.66,4.79,4.91,5.01,5.05,5.25,5.21,5.21,5.21,5.05,4.81,4.56
216,Pattern_Trend,48,0.70,0.10,1,5,0.85,0.75,16,0,32,1.557283,3.380417,17.833067,108.173333,1.3049,1.5376,1.6005,1.6270,1.5878,1.4521,1.5743,1.5738,1.6455,1.6345,1.7727,1.7421,1.7801,1.6519,1.5865,1.5923,1.5910,1.5542,1.4732,1.4464,1.3977,1.3990,1.3905,1.4592,0.46,0.87,1.24,1.63,2.00,2.34,2.79,3.18,3.51,3.79,4.07,4.26,4.33,4.40,4.42,4.45,4.40,4.78,4.63,4.50,4.33,4.00,3.59,3.16
217,Pattern_Trend,48,0.70,0.15,1,5,0.85,0.75,16,0,43,1.705375,3.235000,30.331125,139.105000,1.3265,1.6522,1.9178,1.9444,1.9003,1.8058,1.9101,1.8027,1.7799,1.7416,1.8333,1.7973,1.8323,1.8445,1.7853,1.6887,1.6504,1.6229,1.5612,1.5405,1.5005,1.4887,1.4799,1.5222,0.35,0.74,1.15,1.56,1.94,2.30,2.74,3.13,3.44,3.69,3.91,4.07,4.15,4.21,4.23,4.27,4.24,4.48,4.36,4.24,4.11,3.83,3.45,3.05
182,Pattern_Trend,48,0.60,0.20,1,5,0.85,0.75,16,0,74,1.760742,2.982083,56.294883,220.674167,1.3136,1.6369,1.8616,1.8390,1.8282,1.9430,2.0232,1.9885,1.9727,1.9059,1.9488,1.9154,1.8967,1.8692,1.8151,1.7524,1.7279,1.6812,1.6341,1.6352,1.5371,1.4826,1.5095,1.5400,0.21,0.51,0.78,1.08,1.37,1.68,2.11,2.52,2.86,3.15,3.42,3.66,3.82,3.92,4.00,4.08,4.11,4.26,4.23,4.23,4.21,4.04,3.79,3.53
198,Pattern_Trend,48,0.65,0.10,1,5,0.85,0.75,16,0,42,1.629263,2.981250,26.429025,125.212500,1.3137,1.5595,1.6525,1.6742,1.6681,1.5516,1.6453,1.7358,1.7721,1.7351,1.8461,1.8171,1.8402,1.7121,1.6535,1.6637,1.6572,1.5850,1.5128,1.4854,1.4520,1.4802,1.5199,1.5692,0.37,0.71,1.00,1.28,1.56,1.86,2.25,2.63,2.95,3.23,3.51,3.73,3.82,3.89,3.93,3.96,3.92,4.19,4.11,4.05,3.99,3.81,3.55,3.25
199,Pattern_Trend,48,0.65,0.15,1,5,0.85,0.75,16,0,53,1.742604,2.947917,39.358021,156.239583,1.3301,1.6525,1.9198,1.9409,1.9287,1.8458,1.9280,1.9060,1.8628,1.8059,1.8821,1.8482,1.8721,1.8646,1.8106,1.7301,1.6923,1.6363,1.5792,1.5571,1.5276,1.5377,1.5664,1.5977,0.30,0.64,0.98,1.29,1.61,1.93,2.32,2.71,3.02,3.26,3.49,3.68,3.79,3.84,3.87,3.92,3.89,4.08,4.00,3.94,3.88,3.71,3.45,3.15
163,Pattern_Trend,48,0.55,0.15,1,5,0.85,0.75,16,0,77,1.691900,2.769167,53.276300,213.225833,1.2965,1.6334,1.8108,1.7913,1.7779,1.8661,1.8942,1.8487,1.8469,1.7590,1.7979,1.7751,1.8248,1.7945,1.7400,1.6862,1.6555,1.6103,1.5620,1.5523,1.5170,1.5036,1.5280,1.5336,0.28,0.62,0.95,1.27,1.53,1.83,2.19,2.54,2.85,3.08,3.28,3.46,3.59,3.68,3.75,3.80,3.78,3.88,3.79,3.73,3.63,3.35,2.99,2.61
162,Pattern_Trend,48,0.55,0.10,1,5

In [2]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
75,Pattern_Trend,48,0.6,0.1,1,5,0.85,0.75,16,0,52,1.773813,3.930417,40.238250,204.381667,1.3216,1.6495,1.7736,1.7501,1.7265,1.8634,1.9259,1.9853,2.0126,1.9579,2.0429,2.0072,2.0140,1.8585,1.7828,1.7903,1.7822,1.7074,1.6353,1.6206,1.5557,1.5610,1.6032,1.6440,0.38,0.79,1.16,1.52,1.87,2.27,2.77,3.25,3.66,4.02,4.39,4.69,4.88,5.03,5.18,5.28,5.31,5.59,5.57,5.58,5.58,5.42,5.19,4.95
77,Pattern_Trend,48,0.6,0.1,1,5,0.85,0.85,16,0,30,1.948350,3.836250,28.450500,115.087500,1.3467,1.9139,2.1199,2.0425,2.0669,2.3639,2.3210,2.2227,2.2728,2.2548,2.2491,2.1640,2.0592,1.8749,1.7753,1.8097,1.8368,1.7512,1.6632,1.7140,1.6949,1.7216,1.7721,1.7493,0.32,0.74,1.09,1.46,1.83,2.28,2.81,3.29,3.73,4.14,4.52,4.80,4.96,5.02,5.04,5.03,4.96,5.37,5.29,5.30,5.33,5.21,4.94,4.61
85,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.75,16,0,47,1.884967,3.787500,41.593433,178.012500,1.4951,1.8284,1.8967,1.9268,1.8988,2.0869,2.1090,2.1774,2.2076,2.1269,2.1149,2.0660,2.0559,1.8853,1.7945,1.8174,1.8382,1.7828,1.6959,1.7002,1.6345,1.6493,1.7013,1.7494,0.33,0.71,1.03,1.38,1.71,2.09,2.55,2.99,3.39,3.73,4.05,4.37,4.58,4.74,4.92,5.06,5.19,5.25,5.33,5.49,5.60,5.58,5.47,5.36
83,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.65,16,0,61,1.691183,3.607083,42.162183,220.032083,1.3558,1.4853,1.7293,1.6949,1.7136,1.8712,1.8705,1.9237,1.9071,1.8787,1.8940,1.8540,1.8229,1.7150,1.6701,1.6816,1.7146,1.6625,1.5834,1.5571,1.4902,1.4777,1.5022,1.5330,0.40,0.80,1.19,1.53,1.87,2.20,2.62,3.01,3.36,3.71,4.03,4.33,4.51,4.65,4.72,4.83,4.91,4.93,4.94,4.99,5.01,4.89,4.68,4.46
81,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.55,16,0,62,1.569938,3.470417,35.336125,215.165833,1.3095,1.4464,1.5755,1.5928,1.5714,1.7179,1.6400,1.6973,1.7057,1.6879,1.7069,1.6671,1.6454,1.5666,1.5397,1.5576,1.5917,1.5782,1.5406,1.5152,1.4499,1.4367,1.4568,1.4817,0.44,0.91,1.32,1.69,2.01,2.33,2.68,3.03,3.35,3.65,3.92,4.17,4.31,4.41,4.44,4.50,4.55,4.58,4.60,4.66,4.69,4.57,4.35,4.13
84,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.70,16,0,54,1.637308,3.412083,34.414650,184.252500,1.3875,1.5864,1.7652,1.7337,1.7209,1.8542,1.8180,1.8666,1.8498,1.7999,1.7952,1.7490,1.7209,1.6123,1.5499,1.5628,1.5805,1.5438,1.4787,1.4765,1.4329,1.4412,1.4670,1.5025,0.40,0.83,1.20,1.53,1.86,2.21,2.64,3.01,3.34,3.62,3.88,4.14,4.28,4.37,4.46,4.52,4.57,4.56,4.56,4.61,4.61,4.48,4.23,3.98
82,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.60,16,0,60,1.515296,3.361250,30.917750,201.675000,1.3136,1.4485,1.5474,1.5340,1.5212,1.6740,1.5888,1.6451,1.6549,1.6377,1.6382,1.6004,1.5816,1.5006,1.4681,1.4861,1.5191,1.5065,1.4713,1.4487,1.3871,1.3761,1.3975,1.4206,0.45,0.91,1.32,1.69,2.00,2.31,2.65,3.00,3.31,3.61,3.86,4.10,4.22,4.30,4.31,4.35,4.38,4.39,4.39,4.44,4.45,4.32,4.08,3.83
87,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.85,16,0,29,2.120171,3.246250,32.484954,94.141250,1.4119,2.0285,2.2190,2.2747,2.2985,2.6518,2.5759,2.4747,2.5199,2.4937,2.4788,2.3748,2.2447,2.0258,1.9175,1.9546,1.9824,1.8823,1.7807,1.8354,1.8138,1.8441,1.9026,1.8980,0.25,0.61,0.90,1.24,1.59,2.01,2.47,2.89,3.29,3.66,3.99,4.26,4.43,4.47,4.52,4.50,4.42,4.28,4.20,4.20,4.20,4.09,3.85,3.59
80,Pattern_Trend,48,0.6,0.1,1,5,0.90,0.50,16,0,64,1.570504,

In [2]:
# window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,16,0,61,1.796779,4.061250,48.603529,247.736250,1.5536,1.7174,1.9600,1.8401,1.7683,1.7347,1.7602,1.8465,2.0150,1.9601,1.9061,1.8635,1.9831,1.9197,1.9143,1.8917,1.8577,1.8374,1.7749,1.7487,1.5721,1.5175,1.5784,1.6017,0.29,0.61,0.96,1.32,1.66,2.06,2.58,3.07,3.50,3.90,4.20,4.56,4.82,5.07,5.33,5.53,5.69,6.02,6.05,6.17,6.25,6.16,5.95,5.72
3,Pattern_Trend,48,0.6,0.1,1,5,0.85,0.75,16,0,52,1.773813,3.930417,40.238250,204.381667,1.3216,1.6495,1.7736,1.7501,1.7265,1.8634,1.9259,1.9853,2.0126,1.9579,2.0429,2.0072,2.0140,1.8585,1.7828,1.7903,1.7822,1.7074,1.6353,1.6206,1.5557,1.5610,1.6032,1.6440,0.38,0.79,1.16,1.52,1.87,2.27,2.77,3.25,3.66,4.02,4.39,4.69,4.88,5.03,5.18,5.28,5.31,5.59,5.57,5.58,5.58,5.42,5.19,4.95
1,Pattern_Trend,12,0.6,0.1,1,5,0.85,0.75,16,0,47,2.233446,3.052500,57.971954,143.467500,1.8895,2.1522,2.4347,2.1636,2.0919,2.4483,2.4617,2.3823,2.4664,2.3590,2.2681,2.2116,2.3630,2.2926,2.2370,2.1933,2.1891,2.3207,2.2352,2.2015,2.0711,2.0220,2.0803,2.0676,0.20,0.49,0.79,1.12,1.39,1.71,2.08,2.43,2.70,2.95,3.13,3.33,3.58,3.82,4.02,4.13,4.26,4.59,4.56,4.59,4.60,4.49,4.27,4.03
4,Pattern_Trend,72,0.6,0.1,1,5,0.85,0.75,16,0,56,1.453750,1.606667,25.410000,89.973333,1.2217,1.3817,1.4078,1.4130,1.4396,1.5859,1.6429,1.5169,1.5546,1.4954,1.5479,1.5216,1.5923,1.4997,1.4574,1.4489,1.4447,1.4090,1.3581,1.3667,1.3495,1.3787,1.4197,1.4363,0.33,0.58,0.81,1.06,1.24,1.49,1.75,1.90,1.99,2.07,2.12,2.12,2.09,2.03,1.99,1.95,1.83,1.90,1.73,1.55,1.37,1.92,1.57,1.17
5,Pattern_Trend,96,0.6,0.1,1,5,0.85,0.75,16,0,59,1.302104,0.713750,17.824146,42.111250,1.2569,1.2901,1.3114,1.3222,1.3729,1.5088,1.6119,1.4597,1.4467,1.3821,1.3667,1.3495,1.3653,1.2991,1.2660,1.2312,1.2193,1.2022,1.1925,1.1647,1.1517,1.1464,1.1736,1.1596,0.24,0.42,0.57,0.74,0.86,1.04,1.23,1.31,1.35,1.35,1.31,1.25,1.14,1.00,0.87,0.75,0.55,0.54,0.35,0.17,-0.03,0.45,0.07,-0.40
0,Pattern_Trend,6,0.6,0.1,1,5,0.85,0.75,16,0,72,1.449671,0.032917,32.376300,2.370000,1.5726,1.4380,1.5496,1.3635,1.2824,1.4477,1.4254,1.4344,1.4759,1.4304,1.4658,1.5504,1.6050,1.6191,1.6009,1.5820,1.5438,1.5589,1.4980,1.4542,1.3027,1.2074,1.1910,1.1930,0.11,0.12,0.14,0.10,0.05,0.08,0.17,0.26,0.32,0.35,0.29,0.27,0.23,0.20,0.19,0.17,0.05,0.08,-0.08,-0.20,-0.42,-0.16,-0.56,-0.97


In [2]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_vol_window,Pattern_vol_q_high,Pattern_vol_q_low,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,2,0,56,1.518688,5.456667,29.046500,305.573333,1.5097,1.6609,1.4773,1.3083,1.2607,1.2585,1.3055,1.3505,1.5136,1.5558,1.5539,1.5623,1.6299,1.6489,1.6637,1.6712,1.6645,1.6701,1.6248,1.6354,1.4977,1.4407,1.4975,1.4871,0.50,1.04,1.55,2.02,2.45,2.94,3.52,4.09,4.60,5.10,5.51,5.93,6.29,6.61,6.94,7.22,7.45,8.00,8.14,8.33,8.42,8.35,8.10,7.86
2,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,6,0,51,1.587387,4.795000,29.956762,244.545000,1.5628,1.6145,1.6449,1.4936,1.4460,1.4243,1.4743,1.5957,1.7268,1.6744,1.6379,1.6494,1.8072,1.7575,1.7567,1.7292,1.6910,1.6688,1.6071,1.5940,1.4434,1.3786,1.3612,1.3580,0.44,0.86,1.31,1.72,2.09,2.50,3.04,3.59,4.06,4.49,4.86,5.29,5.65,5.95,6.24,6.47,6.64,7.10,7.14,7.27,7.34,7.24,7.02,6.77
1,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,4,0,47,1.602021,4.755833,28.294979,223.524167,1.6988,1.6109,1.6309,1.3971,1.3147,1.2725,1.3378,1.4854,1.6480,1.6943,1.6673,1.6871,1.8273,1.8339,1.8547,1.8279,1.7829,1.7611,1.6868,1.6725,1.4970,1.4341,1.4147,1.4108,0.48,0.94,1.38,1.79,2.12,2.51,3.06,3.61,4.05,4.48,4.84,5.26,5.61,5.92,6.19,6.39,6.54,7.04,7.08,7.19,7.23,7.09,6.82,6.52
3,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,8,0,54,1.643275,4.625000,34.736850,249.750000,1.6303,1.6284,1.6427,1.5562,1.5540,1.5367,1.5815,1.7065,1.8543,1.7856,1.7522,1.7456,1.8426,1.7936,1.7937,1.7724,1.7340,1.7084,1.6467,1.6144,1.4325,1.3769,1.3771,1.3723,0.39,0.75,1.14,1.54,1.91,2.34,2.90,3.44,3.92,4.36,4.72,5.15,5.50,5.79,6.08,6.31,6.48,6.89,6.92,7.05,7.12,7.00,6.77,6.53
9,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,20,0,62,1.884550,4.469583,54.842100,277.114167,1.6074,1.7756,2.0272,1.8975,1.8001,1.7741,1.8774,1.9592,2.1253,2.0730,2.0251,1.9778,2.0930,2.0261,2.0191,1.9938,1.9579,1.9363,1.8703,1.8457,1.6593,1.5902,1.6477,1.6701,0.33,0.68,1.06,1.45,1.81,2.25,2.81,3.36,3.84,4.28,4.62,5.00,5.30,5.57,5.85,6.07,6.24,6.59,6.64,6.78,6.89,6.82,6.62,6.41
14,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,30,0,70,1.899004,4.325833,62.930292,302.808333,1.6030,1.6831,1.9760,1.8780,1.7909,1.7805,1.8749,2.0524,2.2082,2.1550,2.1063,2.0312,2.1153,2.0230,2.0158,1.9900,1.9388,1.9418,1.8806,1.8586,1.6840,1.6211,1.6725,1.6951,0.29,0.59,0.95,1.33,1.72,2.18,2.76,3.33,3.82,4.27,4.60,4.97,5.25,5.49,5.73,5.90,6.02,6.32,6.34,6.46,6.58,6.52,6.31,6.09
8,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,18,0,62,1.842500,4.246250,52.235000,263.267500,1.5904,1.7422,1.9767,1.8554,1.7609,1.7374,1.8320,1.9121,2.0755,2.0166,1.9704,1.9261,2.0413,1.9766,1.9705,1.9467,1.9123,1.8916,1.8281,1.8001,1.6206,1.5647,1.6247,1.6471,0.30,0.63,0.99,1.36,1.70,2.12,2.66,3.17,3.62,4.04,4.36,4.73,5.01,5.28,5.57,5.78,5.95,6.31,6.35,6.48,6.58,6.51,6.31,6.10
10,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,22,0,64,1.784087,4.195000,50.181600,268.480000,1.5535,1.6124,1.8627,1.7733,1.6968,1.6780,1.7660,1.8477,2.0059,1.9656,1.9241,1.8832,1.9892,1.9189,1.9139,1.8925,1.8605,1.8412,1.7820,1.7621,1.5858,1.5236,1.5786,1.6006,0.31,0.64,0.99,1.37,1.72,2.14,2.68,3.20,3.66,4.08,4.39,4.75,5.01,5.26,5.52,5.71,5.86,6.18,6.20,6.32,6.41,6.32,6.10,5.86
4,Pattern_Trend,24,0.6,0.1,1,5,0.85,0.75,10,0,56,1.646096,4.143333,3

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '4h'
higher_timeframe = '1d'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'vol_window': [24],
                            'vol_q_high': [0.6],
                            'vol_q_low': [0.1],
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.85],
                            'second_candle': [0.75],
                            },
                'Trend': {
                            'timeperiod': [20], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)